# Project: Wrangling and Analyze Data

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [ ]:
import pandas as pd 
import numpy as np 
import requests 
import os
import tweepy 
from timeit import default_timer as timer
from tweepy import OAuthHandler
import json 

In [ ]:
df_1 = pd.read_csv('twitter-archive-enhanced.csv' , sep = ',')

In [ ]:
df_1.head()

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
url1 = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r1 = requests.get(url1)

In [ ]:
print(r1)

In [ ]:
r1.content

In [ ]:
with open(url1.split('/')[-1] , mode ='wb') as file:
    file.write(r1.content)

In [ ]:
df_2 = pd.read_csv('image-predictions.tsv' , sep ='\t')

In [ ]:
df_2.head()

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
# nous avons ici nos tokens pour nous identifier pour avoir accès à 
# notre API twitter
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''
auth = tweepy.OAuthHandler(consumer_key , consumer_secret)
auth.set_access_token(access_token , access_token_secret)

api = tweepy.API(auth , wait_on_rate_limit=True)


In [ ]:
# voici le code pour scraper les données twitter de weRateDogs

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_1.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [ ]:
# on peut aussi choisir de télécharger le fichier tweet_json 
# comme ceci
url2 = 'https://video.udacity-data.com/topher/2018/November/5be5fb7d_tweet-json/tweet-json.txt'
r2 = requests.get(url2)

In [ ]:
print(r2)

In [ ]:
r2.content

In [ ]:
with open(url2.split('/')[-1] , mode ='wb') as file:
    file.write(r2.content)

In [ ]:
df_3 = pd.read_json("tweet-json.txt", lines = True)

In [ ]:
df_3.head()

## Assessing Data




>de manière programmatique et visuelle des trois datasets , en sortant 8 problèmes de qualité au niveau des données et 2 problème de rangements .

### 1. Problème de qualité
   <ul><li>évaluation visuelle</li> </ul>

le premier dataframme "twitter-archive-enhanced.csv" contient quasiment que des valeurs nulls (NAN) sur les colonnes suivants :
<ol>
    <li>in_reply_to_status_id</li>
    <li>in_reply_to_user_id</li>
    <li>retweeted_status_id</li>
    <li>retweeted_status_user_id</li>
    <li>retweeted_status_timestamp</li>
</ol>

<ul>
     <li>évaluation programmatique</li>
</ul> 

Dans l'évaluation programmatique on peut voir que :
>Dans le premier dataset twitter-archive-enhanced.csv
<ol>
    <li> timestamp est de type  objet(string ou chaine ) alors cela devrait etre de type datetime  </li> 
    <li>nous avons les colonnes in_reply_to_status_id, in_reply_to_user_id , retweeted_status_id , retweeted_status_user_id , retweeted_status_timestamp qui ne contiennent quasiment que des valeurs nulles respectivement 2278,2278,2175,2175, 2175 valeurs nulles ;  nous avons aussi la colonne expanded_urls qui contient 59 valeurs nulles 
    <li> il y'a beaucoup de numerateurs et de dénominateurs qui contient des valeurs incohérentes comme des dénominateurs supérieures à 10 et des numérateurs avec des chiffres très grands qui sont problabement des nombres décimaux , donc des floats  </li> 
   
</ol> 

Dans le second dataset image-predictions.tsv 
<ul>
    <li> nous avons un doublon au niveau des valeurs de la colonne img_url </li>
</ul> 
Dans le troixième dataset tweet-json.txt 
<ul>
    <li> on supprime les retweets , on ne gardera que des tweets , ceux avec des images </li>
</ul> 

## 2. Problème d'ordre (Tidiness) 

<ul>
    <li> évaluation visuelle </li>
</ul>

>le premier dataset enfreins à la premiére règle d'ordre "chaque variable doit constituer une colonne " , dans la colonne timestamp on a la date , l'heure et le mois dans la meme colonne or on devrait les séparer en ayant une colonne date , une colonne heure et une colonne mois

<ul>
    <li>évaluation programmatique</li> 
     <li> les étapes du chien doggo , fluffer , pupper , puppo doivent une seule colonne  pour savoir si les chiens sont à quelle étape</li>
</ul> 
toutes les tableauxs doivent former un seul tableau 

In [ ]:
df_1.info()

In [ ]:
df_2.info()

In [ ]:
df_3.info()

In [ ]:
df_1.isnull().sum()

In [ ]:
df_1.rating_numerator.sort_values() 

In [ ]:
df_1.rating_denominator.sort_values()

In [ ]:
df_1.rating_denominator.value_counts()

In [ ]:
df_1.rating_numerator.value_counts()

In [ ]:
df_2.jpg_url.duplicated().sum()

In [ ]:
all_columns = pd.Series(list(df_1) + list(df_2) + list(df_3))
all_columns[all_columns.duplicated()] 

In [ ]:
df_1.describe()

In [ ]:
df_2.describe()

In [ ]:
df_3.describe()

## Cleaning Data

>Avant de procéder au nettoyage faisons d'abord des copies de chacun des ensembles de ces datasets 

In [ ]:
# faisons la copie de nos datasets 
cp_1 = df_1.copy() 
cp_2 = df_2.copy()
cp_3 = df_3.copy()


## Nettoyage des données manquantes 

>les données manquantes que nous avions évalué  dans note premier dataset ""précédemment ne pas utiles dans notre analyse de données 
essayons de les supprimer 

### Define

> Supprimons les colonnes  in_reply_to_status_id, in_reply_to_user_id , retweeted_status_id , retweeted_status_user_id , retweeted_status_timestamp 
qui ne seront pas utiles à notre analyse car contient beaucoup de valeurs manquantes

### Code 

In [ ]:
#calulons le pourcentage des données manquants
prtc_manquants = cp_1.isnull().sum()*100/cp_1.shape[0] 
prtc_manquants

In [ ]:
#suppression des colonnes avec les données manquantes 
cp_1.drop( ["in_reply_to_user_id",
          "retweeted_status_timestamp", 
          "retweeted_status_id",
          "retweeted_status_user_id", 
          "expanded_urls"], axis = 1,  inplace = True)

### Test

In [ ]:
list(cp_1)

### Nettoyage des colonnes inutiles dans le dataframme cp_3

### Define

>Supprimmons les colonnes inutiles dans le dataset tweet_json

### Code

In [ ]:
# suppression des colonnes inutiles
cp_3.drop(["id_str" , "truncated" , "display_text_range" , "display_text_range" ,"entities" , "extended_entities" , "source",
        "in_reply_to_status_id_str" , "in_reply_to_user_id" , 
         "in_reply_to_user_id_str" , "in_reply_to_screen_name" , "user" , "geo" , "coordinates" , "place" , "contributors" ,
         "is_quote_status" , "favorited" , "retweeted" , "possibly_sensitive" , "possibly_sensitive_appealable" ,
         "lang" , "quoted_status_id" , "quoted_status_id_str","quoted_status" ] , axis = 1  , inplace = True)

### Test

In [ ]:
list(cp_3)

### Suppression des colonnes inutiles dans le dataframme image_predictions

### Define 

>Nous allons supprimer les colonnes inutiles du dataset image_prediction

### Code

In [ ]:
#suppression des colonnes inutiles 
cp_2.drop(['img_num',
                  'p2', 
                  'p2_conf',
                  'p2_dog', 
                  'p3',
                  'p3_conf',
                  'p3_dog'] ,axis = 1,  inplace = True )

### Test

In [ ]:
list(cp_3)

### nettoyage des incohérences au niveau des numérateurs et dénominateurs

### Define

>On crééra une colonne moyenne afin d'éviter les nombres incohérents 


### Code

In [ ]:
 #Creons  une  colonne  moyenne  en float qui calculera la moyenne des notations afin de faciliter les calculs .
cp_1['moyenne'] = 10 * cp_1['rating_numerator'] / cp_1['rating_denominator'].astype(float)

### Test

In [ ]:
cp_1.info()


In [ ]:
cp_1.sample(2)

### Nettoyage données dupliquées dans le dataset "image-predictions.tsv"

### Define

> Nettoyage données dupliquées dans le dataset "image-predictions.tsv"

### Code

In [ ]:
#Supprimer l'image dupliquée
cp_2 = cp_2.drop_duplicates(subset=['jpg_url'], keep='last')

###  Test

In [ ]:
cp_2.jpg_url.duplicated().sum()

## Nettoyage ordre  

>Corrigeons le premier problème d'ordre trouvée sur l'évaluation visuelle 
Nous corrigerons aussi le second problème d'ordre qui est de former un seul tableau 

### Define 
> Extrayons la date et l'heure dans deux colonnes différentes dans la colonne timestamp  
On convertit d'abord timestamp au format date puis on fait les séparations

### Code

In [ ]:
#convertissons timestamp au format date
cp_1['timestamp'] = pd.to_datetime(cp_1['timestamp'])

#extraire la date , le mois et l'année dans de nouvelles colonnes 
cp_1['date'] = cp_1['timestamp'].dt.date
cp_1['hour'] = cp_1['timestamp'].dt.time
cp_1['day'] = cp_1['timestamp'].dt.day

#supprimer la colonne timestamp
cp_1.drop( ["timestamp"], axis = 1 , inplace = True)

### Test

In [ ]:
list(cp_1)

### Résolution des colonnes doggo , floofer , puppo et pupper 

### Define

>nous allons fusionner les colonnes doggo , floofer,puppo et pupper dans un meme colonne nommé niveau chien "level_dog"

### Code

In [ ]:
#les données sur le type de chien est None, nous le changerons en  null et nous combinerons les 4 colonnes en une
cp_1 = cp_1.replace('None', np.nan) 
#Combinons les colonnes et supprimons le dernier 
cp_1['dog_level'] = cp_1.doggo.combine_first(cp_1.floofer)
cp_1['dog_level'] = cp_1.dog_level.combine_first(cp_1.pupper)
cp_1['dog_level'] = cp_1.dog_level.combine_first(cp_1.puppo)
cp_1.drop(['doggo' , 'floofer' , 'pupper' , 'puppo'] , axis=1 , inplace = True)

### Test

In [ ]:
cp_1.head()

### Fusionnons les dataframmes df_1 , df_2 et df_3 afin qu'il ne forme qu'un seul dataset

### Define

>Dans un premier temps , nous allons fusionner le dataset twitter-archive-enchanced et le dataset image _prediction dans un dataframme df_4 
Enfin nous créerons un dataframme finale nommé df_twitter

### Code

In [ ]:
# créons  un  dataframe qui fusionnera cp_1 et cp_2
cp_4 = pd.merge(cp_1,cp_2, how = 'left', on = ['tweet_id'])
# créons un dataframme final nommé df_twitter qui fusionnera les dataframmes cp_4 et cp_3
df_twitter = pd.merge(cp_4, cp_3 , how = 'right')


### Test

In [ ]:
df_twitter.info()

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
## ce fichier est trop volumineux donc entraine des problèmes d'espace pour cela on a préféré mis en commentaire 
df_twitter.to_csv('twitter_archive_master.csv', index=False, encoding = 'utf-8')

## Analyzing and Visualizing Data


>Dans cette partie nous sortirons 3 observations et  visualisation de nos données 

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import matplotlib

### 1 ère observation et visualisation

In [ ]:
df_twitter.dog_level.value_counts()

In [ ]:
df_dog_level = df_twitter.groupby('dog_level').filter(lambda x: len(x) >= 25)


In [ ]:
df_dog_level['dog_level'].value_counts().plot(kind = 'barh')
plt.title('Histogramme des niveau de chiens les plus présents dans le sondage')
plt.xlabel('Nombre')
plt.ylabel('les niveaux de chiens')

fig = plt.gcf()


### 2 ème observation et visualisation

In [ ]:
plt.scatter(x="retweet_count", y="moyenne", data=df_twitter)
plt.xlabel('Nombre de retweets')
plt.ylabel('Moyenne')
plt.title('Nombre de retweets par moyenne')

### 3 ième observation

In [ ]:
plusgrand_ind_moy = df_twitter['moyenne'].idxmax()
plus_grand_moy = pd.DataFrame(df_twitter.loc[plusgrand_ind_moy,:])
plus_grand_moy

In [ ]:
pluspetit_ind_moy = df_twitter['moyenne'].idxmin()
pluspetit_moy = pd.DataFrame(df_twitter.loc[pluspetit_ind_moy,:])
pluspetit_moy

### 4 ième observation

In [ ]:
df_5 = df_twitter.copy()
df_5.drop(df_5.index[5:],inplace =True )

In [ ]:
print('les 5 chiens les plus aimés sont :')
for index, row in df_5.iterrows():
    print("chien :", index+1)
    print("Son nom est:", row['name']),
    print("il a ", row['favorite_count'], "J'aime")
    print("il a ", row['retweet_count'] , "retweets")
    print("il est au stade :", row['dog_level'])
    print("Voici comment on le présente:", row['text'])
    print("voici le lien de son image:", row['jpg_url']) 
    print(                           )